# RAG - Retrieval Augmented Generation

## Required Packages

1. **langchain** for orchestration
2. **openai** for the embedding model and LLM
3. **weaviate-client** for the vector database

In [1]:
# Prepare Question
question = 'how can langsmith help with testing?'

In [2]:
# Imports
from dotenv import load_dotenv
import os

In [3]:
# Load Environment variables
load_dotenv()

True

## Preparation

### Collect and load data

In [4]:
from langchain.document_loaders import WebBaseLoader

url = "https://raw.githubusercontent.com/langchain-ai/langchain/master/docs/docs/modules/state_of_the_union.txt"
loader = WebBaseLoader(url)
documents = loader.load()
len(documents)

1

### Chunk Documents

In [5]:
from langchain.text_splitter import CharacterTextSplitter
text_splitter = CharacterTextSplitter(chunk_size=500, chunk_overlap=50)
chunks = text_splitter.split_documents(documents)
len(chunks)

90

### Embed and store the chunks

In [6]:
# from langchain.embeddings import OpenAIEmbeddings
from langchain_openai.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Weaviate
import weaviate 

client = weaviate.connect_to_embedded()

Started /Users/tugbay/.cache/weaviate-embedded: process ID 26631


{"action":"startup","default_vectorizer_module":"none","level":"info","msg":"the default vectorizer modules is set to \"none\", as a result all new schema classes without an explicit vectorizer setting, will use this vectorizer","time":"2024-04-18T17:10:49+02:00"}
{"action":"startup","auto_schema_enabled":true,"level":"info","msg":"auto schema enabled setting is set to \"true\"","time":"2024-04-18T17:10:49+02:00"}
{"level":"info","msg":"No resource limits set, weaviate will use all available memory and CPU. To limit resources, set LIMIT_RESOURCES=true","time":"2024-04-18T17:10:49+02:00"}
{"level":"warning","msg":"Multiple vector spaces are present, GraphQL Explore and REST API list objects endpoint module include params has been disabled as a result.","time":"2024-04-18T17:10:49+02:00"}
{"action":"grpc_startup","level":"info","msg":"grpc server listening at [::]:50050","time":"2024-04-18T17:10:49+02:00"}
{"action":"restapi_management","level":"info","msg":"Serving weaviate at http://12

In [7]:
vectorstore = Weaviate.from_documents(
    client = client,    
    documents = chunks,
    embedding = OpenAIEmbeddings(),
    by_text = False
)

/Users/tugbay/Projects/ai/langchain-learn/.venv/lib/python3.12/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.embeddings.openai.OpenAIEmbeddings` was deprecated in langchain-community 0.0.9 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(
{"level":"info","msg":"Created shard langchain_67e53d2cc73a4662ad80fa437b16dcfd_VWpKMhy0VSaH in 1.537666ms","time":"2024-04-18T16:53:34+02:00"}
{"action":"hnsw_vector_cache_prefill","count":1000,"index_id":"main","level":"info","limit":1000000000000,"msg":"prefilled vector cache","time":"2024-04-18T16:53:34+02:00","took":58291}
/Users/tugbay/Projects/ai/langchain-learn/.venv/lib/python3.12/site-packages/pydantic/main.py:1051: PydanticDeprecatedSince20: The `dict` method is deprec

## Execution

### Step 1: Retrieve

In [8]:
retriever = vectorstore.as_retriever()

### Step 2: Augment

In [9]:
from langchain.prompts import ChatPromptTemplate

template = """You are an assistant for question-answering tasks. 
Use the following pieces of retrieved context to answer the question. 
If you don't know the answer, just say that you don't know. 
Use three sentences maximum and keep the answer concise.
Question: {question} 
Context: {context} 
Answer:
"""
prompt = ChatPromptTemplate.from_template(template)

print(prompt)

input_variables=['context', 'question'] messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], template="You are an assistant for question-answering tasks. \nUse the following pieces of retrieved context to answer the question. \nIf you don't know the answer, just say that you don't know. \nUse three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:\n"))]


### Step 3: Generate

In [11]:
# from langchain.chat_models import ChatOpenAI -- deprecated
from langchain_openai import ChatOpenAI

from langchain.schema.runnable import RunnablePassthrough
from langchain.schema.output_parser import StrOutputParser

llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

rag_chain = (
    {"context": retriever,  "question": RunnablePassthrough()} 
    | prompt 
    | llm
    | StrOutputParser() 
)

query = "What did the president say about Justice Breyer"
rag_chain.invoke(query)

/Users/tugbay/Projects/ai/langchain-learn/.venv/lib/python3.12/site-packages/pydantic/main.py:1051: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.7/migration/
  warnings.warn('The `dict` method is deprecated; use `model_dump` instead.', category=PydanticDeprecatedSince20)


"The president honored Justice Breyer for his service and dedication to the country. He mentioned nominating Judge Ketanji Brown Jackson to continue Justice Breyer's legacy of excellence. The president highlighted Justice Breyer's background and the importance of nominating someone to the Supreme Court."